# A story of weird behavior of Widgetasitc.

## The traceback

We got tracebacks like that:
```
--- Logging error ---
Traceback (most recent call last):
    
...

  File "/home/jhenner/work/widgetastic.patternfly/src/widgetastic_patternfly/__init__.py", line 348, in __repr__
    return '{}({!r})'.format(type(self).__name__, self.locator)
AttributeError: 'SettingsNavDropdown' object has no attribute 'locator'
Call stack:

...
  
  File "/home/jhenner/work/miq/http_503_everywhere/.cfme_venv3/lib64/python3.7/site-packages/widgetastic/browser.py", line 456, in move_to_element
    self.logger.debug
    ('move_to_element: %r', locator)
Unable to print the message and arguments - possible formatting error.
Use the traceback above to help find the error.
```

# What is wrong?
Well it takes some reading effort but the important is:

```AttributeError: 'SettingsNavDropdown' object has no attribute 'locator'```

 * The `SettingsNavDropdown` s a descendant of `widgetastic_patternfly.NavDropdown`.
 * It has no `__repr__` defined, so the `NavDropdown` is what we should look at.

In [1]:
from widgetastic_patternfly import NavDropdown
nav = NavDropdown()
hasattr(nav, 'locator')

False

It trully seem to have no `locator`.

In [2]:
str(NavDropdown())

'NavDropdown()'

Well it works just fine... No error. Or does it really?

In [3]:
NavDropdown().__repr__

<bound method WidgetDescriptor.__repr__ of NavDropdown()>

Hmm nothing specia... **wait what?! ... `WidgetDescriptor`**? I've just created **`NavDropDown`!** ... Or didn't I ?

In [4]:
type(NavDropdown())

widgetastic.widget.base.WidgetDescriptor

I have checked the `NavDropdown` soure. There was **nothing suspiccious** about it.

Until I have seen:

```python
Init signature: Widget(*args, **kwargs)
Source:        
class Widget(object, metaclass=WidgetMetaclass):
    """Base class for all UI objects.

    Does couple of things:

        * Ensures it gets instantiated with a browser or another widget as parent. If you create an
          instance in a class, it then creates a :py:class:`WidgetDescriptor` which is then invoked
          on the instance and instantiates the widget with underlying browser.
        * Implements some basic interface for all widgets.
        
    If you are inheriting from this class, you **MUST ALWAYS** ensure that the inherited class
    has an init that always takes the ``parent`` as the first argument. You can do that on your
    own, setting the parent as ``self.parent`` or you can do something like this:        
    
   .. code-block:: python

        def __init__(self, parent, arg1, arg2, logger=None):
            super(MyClass, self).__init__(parent, logger=logger)
            # or if you have somehow complex inheritance ...
            Widget.__init__(self, parent, logger=logger)

```

## The takeaways from the long text:
* Well to get the real thing, I need to put it inside of other `Widget`/`View`.
* There is a param `parent` that has to be set.

In [5]:
from widgetastic_patternfly import NavDropdown, View

class MyView(View):
    widget = NavDropdown()

type(MyView.widget)

widgetastic.widget.base.WidgetDescriptor

 * **Perhaps** I need to instantiate the View
 * I need a `Browser` intance to do it.
 * There were some problems to do it, but today I new I can just use `MagicMock`.

In [6]:
from widgetastic.browser import Browser
from unittest.mock import MagicMock

selenium_mock = MagicMock()
view = MyView(Browser(selenium_mock))
type(view.widget)

widgetastic_patternfly.NavDropdown

**Looks promissing!** We sucessfully created a real `NavDropdown`. Let's try it fails:

```python
view.widget

---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)

~/work/types_talk/.venv/lib/python3.7/site-packages/widgetastic_patternfly/__init__.py in __repr__(self)
    346 
    347     def __repr__(self):
--> 348         return '{}({!r})'.format(type(self).__name__, self.locator)
    349 
    350 

AttributeError: 'NavDropdown' object has no attribute 'locator'
```

**OMG! OMG! OMG! OMG! It did Fail**. Let's fix it using some mokey-patching.

In [7]:
def __repr__(self):
    return f'The fixed {self.__class__}'

NavDropdown.__broken_repr__ = NavDropdown.__repr__ 
NavDropdown.__repr__ = __repr__

class MyView(View):
    widget = NavDropdown()

view = MyView(Browser(MagicMock()))
view.widget

The fixed <class 'widgetastic_patternfly.NavDropdown'>

## What is happening:
 * We already know the `Widget()` doesn't crate a object of class `Widget`, but a `WidgetDescriptor` object.
 * So the real `Widget` instantiation is delayed to the point of **accessing the `WidgetDescriptor` object inside of the `View` object**

I have got the
 * reproducer
 * the fix

# Making a test for `__repr__`
 * There is **lots of `Widget` descendants** defined in the wrapanapi.
 * I should make a test the __str__ and __repr__ of all the classes the library exports.


I need something like that:

```python
from widgetastic_patternfly import View, Button, Kebab, Text, ...
def test_str(browser):
    class TestView(View):
        any_button = Button()  # will pick up first button...
        button1 = Button('Default Normal')
        button2 = Button(title='noText', classes=[Button.PRIMARY])
        kebab = Kebab(...)
        text = Text(...)
        ...
    view = TestView(browser)
    for widget in view.any_button, view.button1, view.button2, ...:
        assert type(str(widget)) is str
```

but better.


 * there is **no guarantee that newly defined class will get tested**
 * easy to miss something.
 * sore fingers from typing

# Reuse the View for all Widgets?
 * There actually is a way to define the Type/Class in run-time.
 * Reusing is dangerous.

 Lets't take a look how can we do this

In [8]:
def make_me_a_class(bases=tuple(), kw={}):
    class InnerClass(*bases, **kw):
        some_attribute = "attribute_value"
    # We can just set the attrs with setattr dynamically...
    return InnerClass

OrdinaryType = make_me_a_class((View,))

We just created a type/class well let's say **in bit more dynamic way**.


In [9]:
DynamicType = type('TheTestView',
                   (View,),
                   dict(some_attribute="attribute value"))
DynamicType

widgetastic.widget.base.TheTestView

In [10]:
import types

TypesType = types.new_class("TypesTestClass",
                            bases=(View,),
                            kwds=None,
                            exec_body=lambda ns: ns.update(dict(some_attribute="attribute_value")))
TypesType

widgetastic.widget.base.TypesTestClass

So there are **three ways** to create a new type/class in Python. Are there equivalent?

In [11]:
OrdinaryType.__module__, DynamicType.__module__, TypesType.__module__

('__main__', 'widgetastic.widget.base', 'widgetastic.widget.base')

 This is just a small difference of the behavior that could bite some day, but nothing too important. **Results can be pretty much the same**


# Getting all the Widgets
To do that, I used `inspect`.

In [12]:
import widgetastic_patternfly as wp
from widgetastic_patternfly import Widget
from contextlib import suppress
import inspect

DUMMY = "dummy"

# For each Widget, I predefined the values for it's __init__
init_values = {
    wp.AggregateStatusCard: dict(name=DUMMY),
    wp.AggregateStatusMiniCard: dict(name=DUMMY, locator=DUMMY),
    wp.BarChart: dict(id=DUMMY),
    wp.BootstrapNav: dict(locator=DUMMY),
    wp.NavDropdown: dict()
}


widget_classes = {
    name: cls for name, cls
    in inspect.getmembers(wp)
    if inspect.isclass(cls) and issubclass(cls, Widget)  # Widget's descendants only. 
}

widget_descriptors = {
    name: cls(**init_values.get(cls, {}))
    for name, cls in widget_classes.items()
}

TestView = type('TheTestView',
                (View,),
                widget_descriptors)

the_view = TestView(Browser(MagicMock()))

for name in the_view.widget_names:
    with suppress(TypeError, ValueError, NameError):
        print(getattr(the_view, name))
        assert isinstance(widget, Widget)
        print(str(widget))

<Accordion 'Accordion'>
BaseInput(locator=None)
BootstrapNav('dummy')
Button()
<TabWithDropdown 'Generictabwithdropdown'>
Input(locator=None)
The fixed <class 'widgetastic_patternfly.NavDropdown'>
<Tab 'Tab'>
<TabWithDropdownDefault 'Tabwithdropdown'>
TextInput(locator=None)
